# [180. Consecutive Numbers](https://leetcode.com/problems/consecutive-numbers/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Logs

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| num         | varchar |
+-------------+---------+</pre>
In SQL, id is the primary key for this table.
id is an autoincrement column.


Find all numbers that appear at least three times consecutively.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Logs table:
<pre>+----+-----+
| id | num |
+----+-----+
| 1  | 1   |
| 2  | 1   |
| 3  | 1   |
| 4  | 2   |
| 5  | 1   |
| 6  | 2   |
| 7  | 2   |
+----+-----+</pre>
Output:
<pre>+-----------------+
| ConsecutiveNums |
+-----------------+
| 1               |
+-----------------+</pre>
Explanation: 1 is the only number that appears consecutively for at least three times.

In [3]:
#pandas schema

import pandas as pd

data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})

# to spark schema
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

logs_df = spark.createDataFrame(logs)
logs_df.show(truncate=False)

+---+---+
|id |num|
+---+---+
|1  |1  |
|2  |1  |
|3  |1  |
|4  |2  |
|5  |1  |
|6  |2  |
|7  |2  |
+---+---+



In [7]:
# in Spark Dataframe
from pyspark.sql.window import Window

win = Window.orderBy('id')

logs_df\
    .withColumn('ld',lead('num').over(win))\
    .withColumn('lg',lag('num').over(win))\
    .where('num=ld and ld=lg')\
    .select(col('num').alias('ConsecutiveNums'))\
    .show()


+---------------+
|ConsecutiveNums|
+---------------+
|              1|
+---------------+



In [7]:
# in Spark SQL
logs_df.createOrReplaceTempView('logs')
spark.sql('''
select distinct num as ConsecutiveNums
from
(select num,
    lead(num) over(order by id) as ld,
    lag(num) over(order by id) as lg
from logs) x
where num=ld and ld=lg
''').show()

+---------------+
|ConsecutiveNums|
+---------------+
|              1|
+---------------+



In [ ]:
spark.stop()